# 1. Advanced Machine Learning Computer Vision Project: Summary
### Goal: Auto-analysis of the color of half a dozen photos of thousands of ice crystals in snow to determine the angle of the crystals


### Challenge 1: Must assign the same crystal ID to each crystal in each photo. Some crystals disappear (go dark) in some photos
### Challenge 2: Only 3 samples of crystals consisting of 6-8 images each are available; is training an ML model possible?

# 2. Procedure:

1. Align the photos: rotation (PIL; Irfanview) and registration (DRMIME_2D from ZeroCostDL4Mic)

2. Identify crystals of ice across the images with Machine Learning: two methods were tested
    - Method 1: Apply Holistically Nested Edge Detection (HED)  to find edges of the crystals  (from Xie at UCSD: a UNET neural network implemented in Caffe)
    - Then apply Connected Component Analysis (OpenCV) to transform edges into regions of interest (segmentation)
    <br><br>
    - Method 2: cellpose segmentation for microbiology (cytoplasm model) from ZeroCostDL4Mic
    <br><br>

3. Prepare a dataframe of all crystals in all photos with "regionsprops" function in skimage

4. Compare the crystals' refraction color to a color chart to determine wavelength shift from light refraction and to calculate the crystals' angle

# 3. Issues encountered in the project

### ML is pretty good at registering images with linear displacements but not rotation
### I didn't quickly understand pytorchvision methods to detect edges and segment objects
### It was much quicker to apply out-of-the-box pre-trained models for segmentation, edge-detection and component analysis
### These models are sensitive to the contrast of the input image (gamma adustments)
### I learned many ways to manipulate image pixels
### Care must be taken with the different index conventions for color images: pytorch tensor, jpg-tiff, Open CV BGR and YX, PIL and matplotlib!!
### RGB and wavelength of light is a complicated nonlinear mapping. Comparing colors across images requires calibrating the RGB conversion of the images during the data gathering process!!
### OpenCV is tricky: "regioprops" adds a zero-region for the background and the index of each segment (and number of segments) increases by one relative to the filter used in the component threshold layer !!

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import imageio.v3 as iio
from PIL import Image

import os

In [ ]:
# set path containing data folder or use default for Colab (/gdrive/My Drive)
local_folder = "../"
import urllib.request
urllib.request.urlretrieve('https://raw.githubusercontent.com/guiwitz/MLCV/main/notebooks/check_colab.py', 'check_colab.py')
from check_colab import set_datapath
colab, datapath = set_datapath(local_folder)

### Set up experiment series and base directory to work in

In [ ]:
archive = "data/SnowProject/all/"
experiment = "L7"

In [ ]:
print(datapath / archive / experiment / "P2024894.JPG")

In [ ]:
titlepage=plt.imread("../project/Colbeck_Equation_slide.jpg")
plt.figure(figsize=(18,18))
plt.axis("off")
plt.imshow(titlepage)

# Snow crystals (Type 1 ice formed at atmospheric pressure) are anisotropic: stacked sheets of tetrahedrons (H2O)
## The sheets form an optical C-axis across which the crystals also shear more easily.
## Thus optical and mechanical characteristics coincide, providing a possible observation window into fracture mechanics
## Can we find evidence of crystals rolling until they shear along the C-axis?

In [ ]:
fieldwork=plt.imread("../project/track_sample01_slide.jpg")
plt.figure(figsize=(18,18))
plt.axis("off")
plt.imshow(fieldwork)

# A sample of the ice glaze of a ski track under crossed polaroids.
# The sample was prepared by freezing it to a glass slide and grinding the snow off the back on a precision milling machine
# Photographed between polaroid filters on 35mm slide film at -20C
# Converted into RGB space with a digital SLR camera
# Width of the original photograph is 36mm

In [ ]:
sample_scale=plt.imread("../project/Img0_with_scale_slide.jpg")
plt.figure(figsize=(18,18))
plt.axis("off")
plt.imshow(sample_scale)

# Birefringent refraction
- The speed of light in the crystal is maximum along the "ordinary" C-axis and less along the extraordinary axes.

https://www.microscopyu.com/techniques/polarized-light/principles-of-birefringence


In [ ]:
biref_waves=plt.imread("../project/birefgringent_wavefronts_01.jpg")
plt.figure(figsize=(8,10))
plt.imshow(biref_waves)

# The polarizer
https://www.microscopyu.com/techniques/polarized-light/principles-of-birefringence

In [ ]:
biref_polaroids=plt.imread("../project/birefringent_polaroids_slide.jpg")
plt.figure(figsize=(18,18))
plt.axis("off")
plt.imshow(biref_polaroids)

# The crystal color is black when the azimuth angle of the C-axis or extraordinary beam is aligned with a polariod
# The color is brightest when the azimuth angle of the C-axis or extraordinary beam is aligned 45 deg to a polaroid
# The color at this azimuth uniquely corresponds to the maximum wavefront path difference (nm)
# This path length difference allows calculation of the polar angle of the crystal relative to the light beam

In [ ]:
basepath = datapath / archive / experiment
print(basepath)

### Read all the raw data files (JPG) from the base directory

In [ ]:

im_list =[] # list of raw data images

# iterate through files and add to list of images
for file in os.listdir(os.path.join(basepath, "resized")):
    if file.endswith("jpg"):
        file_path=basepath / "resized" / file
        print(file_path)
    #load_image(file_path)
        im_list.append(Image.open(file_path))

## Animation of the images

In [ ]:
from matplotlib import animation
plt.rcParams["animation.html"] = "jshtml"  # for matplotlib 2.1 and above uses JavaScript

In [ ]:
len(im_list)

In [ ]:

%%capture
fig, ax = plt.subplots(figsize=(10,7))
backimg=im_list[0]
ax.imshow(im_list[0])
img=plt.imshow(backimg)


def animate(i):
    backimg=im_list[i]
    imgnew=img.set_data(backimg)

 
    ax.imshow(backimg)

    return(img)

ani = animation.FuncAnimation(fig, animate, frames = 9)

In [ ]:
ani

# Goal is to identify each crystal and trace its color/intensity shifts through the angle of azimuthal rotation.

### There is one image at each azimuth angle, approx. 6-7 images per snow sample
### The same crystal appears in each photo but it is a different color and at different coordinates!
### It needs to retain the same ID number in each image.


In [ ]:
import cv2
crystal_edges_hand = cv2.imread(str(datapath / "project/L7_1993_crop.jpg"))
fig, ax = plt.subplots(1,1,figsize=(10,7))
#plt.axis("off")
ax=plt.imshow(cv2.cvtColor(crystal_edges_hand, cv2.COLOR_BGR2RGB))

### Once the crystals are labelled, compare their color to a chart to determine the path length difference
### Calculation of the various angles is then trivial

# The birefringence causes destructive interference of wavelengths in white light, yielding Newton's Color Scale

### Use the Michel-Levy chart (1884, 1909, 1930, ...) of this color series to match the color of the crystal with the birefringent wavelength shift
- Source of this chart:   https://www.geological-digressions.com/optical-mineralogy-some-terminology/

# Birefringence interference color chart from Michel-Levy

https://www.geological-digressions.com/optical-mineralogy-some-terminology/


In [ ]:
mlchart=plt.imread("../project/birefringence_chart01_.jpg")
plt.figure(figsize=(20,50))
plt.imshow(mlchart)

In [ ]:
sample_scale=plt.imread("../project/thick_sample01.jpg")
plt.figure(figsize=(10,10))
plt.axis("off")
plt.imshow(sample_scale)

# Procedure

### Start by lining the images up
- The rotation angle is noted in the original data collection notebook and is not always accurate.
- Rotation alone (e.g. PIL rotate or hand rotation in photo editor) does not sufficiently pre-align the images, and translation to the approximate correct alignment is additionally needed.
- Aligning images is called REGISTRATION
- DRMIME-2D image registration works on simple images with few objects
- Deep learning registration is not good at rotating images!
- The best registration result required first rotating and aligning by hand with an editor, i.e. Irfanview


### Once images are registered, identify the same crystals in each image
- This is called SEGMENTATION and first requires MASKING of Regions Of Interest (ROI)
- One mask for ALL images!

### Given segments, find average color in each crystal/segment in each image and look up the path length
- This requires mapping the RGB to a wavelength which is an average over the distribution of wavelengths that physically combine to produce the perceived color. It is a highly nonlinear function
- The intereference colors are non-chromatic
- The Michel-Levy chart and the photographs are not calibrated
- The best possible method for this project is to find the nearest RGB in the Michel-Levy chart for each crystal / segment

### Interpolate where necessary to find the black or brightest angle if azimuth is desired
### Use the max path difference wavelength found in surveying all the images to derive the sample thickness
### Use the sample thickness to calculate the polar angle of each crystal's C-axis
### Plot the 3D histogram of vertical angle and summarize the C-axis angle statistically

# Rotate the images
- approximate angle to rotate was measured in the experiment by projecting the image on the wall and tracing the edge of the slide, then measuring the trace relative to the trace of the first slide
- Rotation of each image must be extremely precise for the unsupervised affine registration to be able to align the images
- Registration is improved a lot by also translating each image by hand to align each as well as possible before attempting unsupervised registration
- Conceivably, one could use ML to discover keypoints in image pairs, pick two matching keypoints in each image, calculate a line between them in each image, and calculate the rotation angle and translation required from those keypoints.
- see ../data/SnowProject/Book1.xls

In [ ]:
#angles = [0.,-16.,-20.,-29.2,-38.5,-51.,-62.5,-74.,-84.5]# L7 recorded 1993
angles = [0.,-16.,-26.,-29,-38.5,-60.,-80,]# L7 Irfanview 2023

In [ ]:
rot_im_list = []
i=0
for i in range(0,len(angles)):#im in im_list:
    rot_im_list.append(im_list[i].rotate(angles[i],resample=0, expand=0, center=None, translate = None, fillcolor=None))
    i+=1

In [ ]:
#nimages=len(rot_im_list)
#fig, ax = plt.subplots(1,nimages, figsize=(15,15));
#for i in range(nimages):
#    ax[i].imshow(rot_im_list[i])    

In [ ]:

%%capture
fig, ax = plt.subplots(figsize=(10,7))
backimg=rot_im_list[0]
ax.imshow(im_list[0])
img=plt.imshow(backimg)


def animate(i):
    backimg=rot_im_list[i]
    imgnew=img.set_data(backimg)

 
    ax.imshow(backimg)
    ax.set_title(angles[i])

    return(img)

ani = animation.FuncAnimation(fig, animate, frames = 7)

In [ ]:
ani

### Save the rotated images as TIFF

In [ ]:
#try:
#    os.makedirs(datapath / archive / experiment / "auto_rotated")
#except FileExistsError:
#    # directory already exists
#    pass
#for i in range(len(rot_im_list)):
#    rot_im_list[i].save((datapath / archive / experiment / "auto_rotated" / f"{i}.TIFF"), "TIFF")


### The angles for 2023 were found by hand with Irfanview (GUI).
- It does not cost much more time with the GUI to also translate the images (crop)
- These pre-processed images were registered very well
- Previous experiments with registration of the auto-rotated images did not work well

# Register the images to image 0 with DRMIME_2D_ZeroCostDL4Mic.ipynb
- I used it on colab
- Lucas von Chamier*, Romain F. Laine*, Johanna Jukkala, Christoph Spahn, Daniel Krentzel, Elias Nehme, Martina Lerche, Sara Hernández-pérez, Pieta Mattila, Eleni Karinou, Séamus Holden, Ahmet Can Solak, Alexander Krull, Tim-Oliver Buchholz, Martin L Jones, Loic Alain Royer, Christophe Leterrier, Yoav Shechtman, Florian Jug, Mike Heilemann, Guillaume Jacquemet, Ricardo Henriques. Democratising deep learning for microscopy with ZeroCostDL4Mic. Nature Communications, 2021. DOI: https://doi.org/10.1038/s41467-021-22518-0
- load the fixed image to data/FixedImage
- load the rotating images to data/MovingImage
- download the result from data/Prediction
- crop the images to an area of interest
- Most recent successful output is from hand-rotation and translation:
C:\Users\jkhac\Documents\AML\Module 6 Computer Vision\data\SnowProject\all\L7\cropped\run3

In [ ]:
# set path containing data folder or use default for Colab (/gdrive/My Drive)
local_folder = "../"
import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/guiwitz/MLCV/main/notebooks/check_colab.py", "check_colab.py")
from check_colab import set_datapath
colab, datapath = set_datapath(local_folder)
archive = "data/SnowProject/all/"
experiment = "L7"
basepath = datapath / archive / experiment
print(basepath)
imgpath = basepath / "cropped/run3_optimized_cellpose_WB"
print(imgpath)

In [ ]:
import os
im_list =[] # list of raw data images

# iterate through files and add to list of images
for file in os.listdir(imgpath):
    if file.endswith("jpg"):
        file_path=imgpath / file
        print(file_path)
    #load_image(file_path)
        im_list.append(cv2.imread(str(file_path)))

In [ ]:
angles = [0.,-16.,-29,-38.5,-60.,-80,]# L7 Irfanview 2023 -- image 2 didn't register well and is omitted

In [ ]:

%%capture
fig, ax = plt.subplots(figsize=(10,7))
backimg=cv2.cvtColor(im_list[0], cv2.COLOR_BGR2RGB)
ax.imshow(backimg)
img=plt.imshow(backimg)


def animate(i):
    backimg=cv2.cvtColor(im_list[i], cv2.COLOR_BGR2RGB)
    imgnew=img.set_data(backimg)

 
    ax.imshow(backimg)
    ax.set_title(-1*angles[i])

    return(img)

ani = animation.FuncAnimation(fig, animate, frames = 6)

In [ ]:
ani

In [ ]:
ax=plt.imshow(cv2.cvtColor(crystal_edges_hand, cv2.COLOR_BGR2RGB))

# Instance segmentation with pytorchvision
- https://pytorch.org/vision/stable/models.html
- https://arxiv.org/abs/1506.01497

I'm not sure what this is supposed to do. The following code opens an Irfanview window with the input image and the python code keeps running without printing "Finished". It finishes with a warning that no boxes were drawn. Is it trained to COCO labels?

# Apply the Holistically Nested Edge Detection (HED) algorithm to make an edge-based mask for identifying and tracking individual crystals through the pre-registered images

Main reference: https://youtu.be/un7QvhXZ_G4

Original HED papr: https://arxiv.org/pdf/1504.06375.pdf


Saining Xie, Zhuowen Tu,Holistically-Nested Edge Detection, Proceedings of the IEEE International Conference on Computer Vision (ICCV), 2015, pp. 1395-1403 


Caffe model is encoded into two files
1. Proto text file: https://github.com/s9xie/hed/blob/master/examples/hed/deploy.prototxt
2. Pretrained caffe model: http://vcl.ucsd.edu/hed/hed_pretrained_bsds.caffemodel
(information current as of October 2022)

Steps for edge detection followed by connected components-based labeling
for object segmentation:
    
1. Define the crop layer (not implemented by default) ​
2. Define the network and load the pre-trained model.​
3. Register the crop layer with the network
4. Create blob from the image – basically create a preprocessed image​
5. Load pretrained model (you need both the proto text and caffe model files)​
6. Pass the blob image through model​ (forward pass)
7. Get output​
8. Get image ready for connected components (blur, threshold)​
9. Perform connected components based labeling​
10. (Optional) draw markers for visualization purposes​
11. (Optional) filter out small objects​
12. Export your data​


In [ ]:
import cv2
#from matplotlib import pyplot as plt
#import numpy as np
import imageio.v3 as iio
from PIL import Image

#import os

#from matplotlib import animation
#plt.rcParams["animation.html"] = "jshtml"  # for matplotlib 2.1 and above uses JavaScript

### There is a Crop layer that the HED network uses which is not implemented by default. To ensure that the crop is the same for each image, we need to provide our own implementation of this layer.
- Without the crop layer, the final result will be shifted to the right and bottom

In [ ]:
class CropLayer(object):
    def __init__(self, params, blobs):
        # initialize our starting and ending (x, y)-coordinates of
        # the crop
        self.startX = 0
        self.startY = 0
        self.endX = 0
        self.endY = 0

    def getMemoryShapes(self, inputs):
        # the crop layer will receive two inputs -- we need to crop
        # the first input blob to match the shape of the second one,
        # keeping the batch size and number of channels
        (inputShape, targetShape) = (inputs[0], inputs[1])
        (batchSize, numChannels) = (inputShape[0], inputShape[1])
        (H, W) = (targetShape[2], targetShape[3])

        # compute the starting and ending crop coordinates
        self.startX = int((inputShape[3] - targetShape[3]) / 2)
        self.startY = int((inputShape[2] - targetShape[2]) / 2)
        self.endX = self.startX + W
        self.endY = self.startY + H

        # return the shape of the volume (we"ll perform the actual
        # crop during the forward pass
        return [[batchSize, numChannels, H, W]]

    def forward(self, inputs):
        # use the derived (x, y)-coordinates to perform the crop
        return [inputs[0][:, :, self.startY:self.endY,
                self.startX:self.endX]]


### The pre-trained model that OpenCV uses has been trained in the Caffe framework
- Download from the link above

In [ ]:
protoPath = "C:/Users/jkhac/Documents/AML/Module 6 Computer Vision/notebooks/hed_model/deploy.prototxt"
modelPath = "C:/Users/jkhac/Documents/AML/Module 6 Computer Vision/notebooks/hed_model/hed_pretrained_bsds.caffemodel"
net = cv2.dnn.readNetFromCaffe(protoPath, modelPath)
#net = cv2.dnn.readNetFromCaffe(modelPath, protoPath)

### Add the crop layer to the model

In [ ]:
cv2.dnn_registerLayer("Crop", CropLayer)

### Set the paths to the input and output images

In [ ]:
# set path containing data folder or use default for Colab (/gdrive/My Drive)
#local_folder = "../"
#import urllib.request
#urllib.request.urlretrieve("https://raw.githubusercontent.com/guiwitz/MLCV/main/notebooks/check_colab.py", "check_colab.py")
#from check_colab import set_datapath
#colab, datapath = set_datapath(local_folder)
#archive = "data/SnowProject/all/"
#experiment = "L7"
#basepath = datapath / archive / experiment
#print(basepath)
#imgpath = basepath / "cropped/run3_optimized_cellpose_WB"
#print(imgpath)

### Load the input image and grab its dimensions to define the blob
- OpenCV can handle a batch of images in a list
- OpenCV stores images as Blue, Green, Red instead of Red, Green, Blue
- OpenCV also switches X and Y

### Read in a list of images for batch processing

In [ ]:
#import os
#im_list =[] # list of raw data images

# iterate through files and add to list of images
#for file in os.listdir(imgpath):
#    if file.endswith("jpg"):
#        file_path=imgpath / file
#        print(file_path)
    #load_image(file_path)
#        im_list.append(cv2.imread(str(file_path)))

In [ ]:
img0=im_list[0]
img0_rgb = cv2.cvtColor(img0, cv2.COLOR_BGR2RGB)
#plt.imshow(img0_rgb)
(H, W) = img0.shape[:2]

In [ ]:
angles = [0.,-16.,-29,-38.5,-60.,-80,]# L7 Irfanview 2023 -- image 2 didn't register well and is omitted

# Segment and label each crystal

### Pre-process: adjust colors
- gamma shift

In [ ]:
def gamma_shift(img,gamma):

#create lookup table
    values = np.arange(0, 256)
    lut = np.uint8(255 * np.power((values/255.0), gamma))

#gamma adjustment. convert image using LUT table. It maps the pixel intensities in the input to the output using values from lut
    return cv2.LUT(img, lut)

# Preprocessing pipeline
- Positive gamma lets the HED algorithm find edges better

In [ ]:
processed_im_list=[]
for image in im_list:
    processed_im_list.append(gamma_shift(image,2.0))

In [ ]:
fig,ax=plt.subplots(1,2,figsize=(15,15))
ax[0].imshow(cv2.cvtColor(im_list[0], cv2.COLOR_BGR2RGB))
ax[1].imshow(cv2.cvtColor(processed_im_list[0], cv2.COLOR_BGR2RGB))

### Construct a blob out of the input images for the DNN: consistent with images used for training the model
- "blob" is a preprocessed image. 
- OpenCV’s deep neural network (dnn ) module contains two functions that can be used for preprocessing images and preparing them for classification in pre-trained deep learning models.
- It includes scaling and mean subtraction
- The parameters to adjust to tune the edge search are "scale factor (sf)" and the "mean" pixel intensities
 -- Higher scale factors give smaller/more segments and a log-log distributed intensity; lower scale factors give bigger/fewer segments and Normal-distributed intensities
- https://pyimagesearch.com/2017/11/06/deep-learning-opencvs-blobfromimage-works/

In [ ]:
#mean_pixel_values= np.average(img0, axis = (0,1))
mean_pixel_values=(np.average(processed_im_list, axis=(0,1,2)))
sf = .5
print(f"Mean pixel values {mean_pixel_values} and scale factor {sf}")
blob = cv2.dnn.blobFromImages(processed_im_list, scalefactor=sf, size=(W, H),
                             mean=(mean_pixel_values[0], mean_pixel_values[1], mean_pixel_values[2]),
                             #mean=(178, 211, 173),
                             #mean = (150,190,150),
                             swapRB= False, crop=False)

In [ ]:
blob.shape

In [ ]:
#garbage=plt.hist(blob[0,1,:,:])

### View image after preprocessing (blob batch)

In [ ]:
nimages=blob.shape[0]
fig, ax = plt.subplots(1,nimages, figsize=(25,25));
for i in range(nimages):
    blob_for_plot = np.moveaxis(blob[i,:,:,:], 0,2)
    ax[i].imshow(blob_for_plot)    

### Set the blob batch as the input to the holistically nested edge detection (HED) network and perform a forward pass to compute the edges of each image
- Result is a single-channel float32 image for each image in the batch

In [ ]:
net.setInput(blob)
hed = net.forward()
#hed = hed[0,0,:,:]  #Drop the other axes 
#hed = cv2.resize(hed[0, 0], (W, H))
#hed = (255 * hed).astype("uint8")  #rescale to 0-255
hed = (255 * hed)  #rescale to 0-255

In [ ]:
hed.shape

In [ ]:
hed.dtype

### Plot the edge masks

In [ ]:
nimages=hed.shape[0]
fig, ax = plt.subplots(1,nimages, figsize=(25,25));
for i in range(nimages):
    hed_for_plot = np.moveaxis(hed[i,:,:,:], 0,2)
    ax[i].imshow(hed_for_plot)    

In [ ]:

%%capture
fig, ax = plt.subplots(figsize=(10,7))
backimg=np.moveaxis(hed[0,:,:,:], 0,2)
ax.imshow(backimg)
img=plt.imshow(backimg)


def animate(i):
    backimg=np.moveaxis(hed[i,:,:,:], 0,2)
    imgnew=img.set_data(backimg)

 
    ax.imshow(backimg)
    ax.set_title(-1*angles[i])

    return(img)

ani = animation.FuncAnimation(fig, animate, frames = 6)

In [ ]:
ani

### Since some crystals are indistinguishable from background in some images, consolidate the batch edge masks to capture all the edges in one mask by summing the pixel values.

In [ ]:
hedsum=hed[0,0,:,:]*0.
for i in range(hed.shape[0]):
    hedsum=np.add(hedsum, hed[i,0,:,:])

In [ ]:
hedsum.shape

### Scale (average) the edge mask values and make them integer

In [ ]:
hedsum=(hedsum/hed.shape[0]).astype("uint8")

In [ ]:
plt.imshow(hedsum)

### Nonlinear scaling to highlight peaks against background

In [ ]:
#plt.imshow(hedsum**3.)
test=hedsum.copy()
test=np.clip((test**1.2),0,255)
test[test<100]=0
plt.imshow(test)

### Histogram of pixel intensities in edge mask and in nonlinear scaled edge mask

In [ ]:
#Flatten image and plot histogram
image3_flat=(hedsum**3.).flatten()
ax=plt.hist(image3_flat,100)

## Histogram of pixel intensities in edge mask

In [ ]:
#Flatten image and plot histogram
image_flat=(hedsum).flatten()
ax=plt.hist(image_flat,100)

In [ ]:
# Find all the pixels in the 100 x 100 image green channel that are less bright than the x-th percentile
test=hedsum**3.
test=test/test.max()
#threshold=np.quantile(test,threshold_val)
threshold = .05# test is scaled 0-1
mask1=test < threshold
mask2=test >= threshold
#print(mask1.shape)
pix_u_threshold=test.copy()
pix_u_threshold[mask1]=255
pix_u_threshold[mask2]=0
pix_u_threshold=pix_u_threshold.astype("uint8")
#print(pix_u_threshold.shape)

### Threshold image to accentuate bright pixels -- compare with threshold filters below

In [ ]:
plt.imshow(pix_u_threshold)


### Connected component based labeling

- Load segmented binary image, Gaussian blur, grayscale, Otsu"s threshold
- Note that Otsu"s threshold is hardly different from slashing out the low and high as above

In [ ]:
#blur = cv2.GaussianBlur(hedsum, (5,5), 0)#hedsum
blur = cv2.bilateralFilter(hedsum,9,5,10)
thresh0 = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
#thresh0 = cv2.threshold(blur, 0, 255, cv2.THRESH_OTSU)[1]
#plt.imshow(thresh)

In [ ]:
plt.imshow(thresh0)

In [ ]:
low=127
high=255
ret,thresh1 = cv2.threshold(blur,low,high,cv2.THRESH_BINARY)
ret,thresh2 = cv2.threshold(blur,low,high,cv2.THRESH_BINARY_INV)
ret,thresh3 = cv2.threshold(blur,low,high,cv2.THRESH_TRUNC)
#ret,thresh4 = cv2.threshold(blur,low,high,cv2.THRESH_TOZERO)
#ret,thresh5 = cv2.threshold(blur,low,high,cv2.THRESH_TOZERO_INV)
thresh4 = cv2.adaptiveThreshold(hedsum,255,cv2.ADAPTIVE_THRESH_MEAN_C,\
            cv2.THRESH_BINARY,11,2)
thresh5 = cv2.adaptiveThreshold(hedsum,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY,11,0)
#titles = ["Original Image","BINARY","BINARY_INV","TRUNC","TOZERO","TOZERO_INV"]
titles = ["Original Image","BINARY","BINARY_INV","TRUNC","ADAPT MEAN","ADAPT GAUSS"]
images = [img0_rgb, thresh1, thresh2, thresh3, thresh4, thresh5]
fig, axs = plt.subplots(2, 3, figsize=(15, 10))
for i in range(6):
    plt.subplot(2,3,i+1),plt.imshow(images[i],"gray",vmin=0,vmax=255)
    plt.title(titles[i])
    #plt.xticks([]),plt.yticks([])
plt.show()

# Perform connected component labeling
## Use a threshold image (0 (background)and 255 (regions of interest))

In [ ]:
thresh=255-thresh5

In [ ]:
#mybins=(np.arange(0,256,1).tolist())
#garbage=plt.hist(thresh,bins=mybins)

In [ ]:
#type(thresh), thresh.shape,thresh.max()

In [ ]:
#(thresh>100).sum()/(thresh.shape[0]*thresh.shape[1])

In [ ]:
n_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(thresh, connectivity=4)#4 just X,Y or 8 with diagonals  -- changes the segmentation but not the end result (size or angle distribution)

# Create false color image with black background and colored objects

In [ ]:
colors = np.random.randint(0, 255, size=(n_labels, 3), dtype=np.uint8)
colors[0] = [0, 0, 0]  # black background
false_colors = colors[labels]
fig,ax=plt.subplots(1,2,figsize=(20,24))
ax[0].imshow(false_colors)
ax[1].imshow(crystal_edges_hand)



In [ ]:
print(n_labels,labels.max())

# Obtain centroids
- instead of drawing centroids, draw the ID of each segment so we can investigate further

In [ ]:
false_colors_centroid = false_colors.copy() # overlay any of the images with the centroid markers, for example rgb_img
for centroid in centroids:
    cv2.drawMarker(false_colors_centroid, (int(centroid[0]), int(centroid[1])),
                   color=(255, 255, 255), markerType=cv2.MARKER_CROSS)
plt.imshow(false_colors_centroid)


# Remove objects that are too big and/or small
- add the segment number to the overlay plot
- remove filtered segments from the false_colors_filtered layer

In [ ]:
MIN_AREA = 9*9
MAX_AREA = 80*80
false_colors_area_filtered = false_colors.copy()
filter_index=[]
for i, centroid in enumerate(centroids[1:], start=1):# centroid[0] is the background
    area = stats[i, 4]
    if area > MIN_AREA and area < MAX_AREA:
        cv2.drawMarker(false_colors_area_filtered, (int(centroid[0]), int(centroid[1])),
                      color=(255, 255, 255), markerType=cv2.MARKER_CROSS)
        cv2.putText(false_colors_area_filtered, str(i), (int(centroid[0]), int(centroid[1])),
                       fontFace=4,fontScale=1.,color=(255, 255, 255))
    else:
       # print(i, area)
        filter_index.append(i) # index of which segments will be filtered out
    
plt.figure(figsize=(12, 10))
plt.subplot(221)
plt.imshow(img0_rgb)
plt.subplot(222)
plt.imshow(hedsum)
plt.subplot(223)
plt.imshow(thresh)
plt.subplot(224)
plt.imshow(false_colors_area_filtered) 
plt.show()


In [ ]:
len(filter_index)

In [ ]:
# Replace labels of the filtered-out regions of interest with 0
false_colors_filtered=false_colors.copy()
for i in range(len(filter_index)):
    false_colors_filtered[labels==filter_index[i]]=0

In [ ]:
# This plot isn't showing expanses of black (color[0]) as expected ?!
plt.imshow(false_colors_filtered)

### Overlay an alpha image of the continuous clusters on the original image0 and/or the centroid crosses

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))
ax.imshow(img0_rgb)
ax.imshow(false_colors_filtered, alpha=.4)
ax.set_axis_off()

In [ ]:

%%capture
fig, ax = plt.subplots(figsize=(10,7))
#fig=plt.figure()
backimg=img0_rgb
ax.imshow(img0_rgb)
ax.imshow(false_colors_filtered, alpha=0.3)
ax.set_axis_off()
img=plt.imshow(backimg)
#img=plt.imshow(img_rgb)


def animate(i):
    backimg=cv2.cvtColor(im_list[i], cv2.COLOR_BGR2RGB)
    imgnew=img.set_data(backimg)

 
    ax.imshow(backimg)
    ax.imshow(false_colors_filtered, alpha=0.3)
    ax.set_axis_off()

    return(img)

ani = animation.FuncAnimation(fig, animate, frames = 6)

## Many crystals easily discerned with the eye were not captured by the edge algorithm
## Some segments contain several crystals and should be individual segments
## Need to check the resulting segments by hand for errors before processing further

In [ ]:
ani


# We can use regionprops from skimage.measures to extract various parameters

- module calculates useful parameters for each object in one or a batch of images.

- first, define the function to convert RGB to path difference (wavelength in nm)

- make a props object for each image in image list
- add features: azimuth angle, path difference wavelength
- drop superfluous variables
- add the coordinates (centroid x,y) to the dataframe --> easiest to do this before filtering
- concatenate and make pandas df

# Use the Michel-Levy chart (1884, 1930) of this color series to match the color of the crystal with the birefringent wavelength shift
- Source of this chart:   https://www.geological-digressions.com/optical-mineralogy-some-terminology/

In [ ]:
mlchart_full = cv2.imread(str(datapath / "project/birefringence_chart01_.jpg"))
fig, ax = plt.subplots(1,1,figsize=(20,100))
plt.axis("off")
ax=plt.imshow(cv2.cvtColor(mlchart_full, cv2.COLOR_BGR2RGB))

# Calculate path length wavelength to derive the birefringence and thus the polar angle
### "Don't trust your eyes" -- know how your sensor and the computer are processing the light
- https://bioimagebook.github.io/chapters/1-concepts/1-images_and_pixels/images_and_pixels.html#chap-pixels

 - Many RGB colors do not have wavelength (mix of wavelengths) --> dominant wavelength
 - Use the Hue as approximation. Find a RGB_to_Hue or HSV converter shorthand
 - Use the "colour" package
 - Sample the Michel-Levy chart until a RGB match is found. Read off the value of wavelength shift.
 - add the path difference wavelength variable to the df


In [ ]:

### Read in the michel-levy chart
# record the BGR values at all pixels at X=[250,1800], Y[750]
# calculate the wavelength linear with X (pixel), Lamdba (nm) => 250,0 and 1744,1755

mlchart = cv2.imread(str(datapath / "project/birefringence_chart01_copy.jpg"))
#mlchart=cv2.cvtColor(mlchart, cv2.COLOR_BGR2RGB)
mlchart.shape

# Note x and y are reversed from jpg!
mlchart_shift=gamma_shift(mlchart,1.)
ml_pixels_row=mlchart_shift[797:803,250:1800,:]
ml_pixels_row.shape

# Look up the color of each crystal on this color chart and match it to the path difference on the X-axis

In [ ]:
spectral_img=cv2.cvtColor(mlchart_shift[797:803,250:1800,:],cv2.COLOR_BGR2RGB)
xx=np.mean(ml_pixels_row, axis=0)
pathdiff_wavelength = np.array(range(0,xx.shape[0]))
pathdiff_wavelength=pathdiff_wavelength*(1744.)/(1754-250)
xx=pathdiff_wavelength
fig, ax = plt.subplots(1,2,figsize=(20,100))
ax[0].imshow(spectral_img, extent=[0, 1796, 0, 20*spectral_img.shape[0]])
ax[0].plot(xx, 0*xx, '-', linewidth=0, color="black")
ax[0].set_xlabel("Path Difference (nm)")
ax[0].set_xlim(0,1800)


ax[1].imshow(cv2.cvtColor(im_list[0], cv2.COLOR_BGR2RGB))

# The colors in the color bar don't seem to be the same hue/saturation as those in the data sample. Match the RGB histogram of the bar to that of the data image

In [ ]:
from skimage.io import imread, imsave
from skimage import exposure
from skimage.exposure import match_histograms

# Load left and right images
L = spectral_img
R = cv2.cvtColor(im_list[0], cv2.COLOR_BGR2RGB)

# Match using the right side as reference
matched = match_histograms(L, R, multichannel=True)

# Place side-by-side and save
#result = np.hstack((matched,R))
fig, ax = plt.subplots(1,2,figsize=(20,100))
ax[0].imshow(matched, extent=[0, 1796, 0, 20*L.shape[0]])
ax[0].plot(xx, 0*xx, '-', linewidth=0, color="black")
ax[0].set_xlabel("Path Difference (nm)")
ax[0].set_xlim(0,1800)


ax[1].imshow(R)

# That didn't look right at all. Don't use it

In [ ]:
ml_pixels_row=np.mean(ml_pixels_row, axis=0) # this is the average of 6 pixels at each wavelength from the mlchart
#ml_pixels_row=np.mean(matched, axis=0) # this is the average of 6 pixels at each wavelength from the histogram-adjusted mlchart
ml_pixels_row.shape

In [ ]:
pathdiff_wavelength = np.array(range(0,ml_pixels_row.shape[0]))
pathdiff_wavelength=pathdiff_wavelength*(1744.)/(1754-250)# read off the chart: calibration of wavelength and pixel number
max_pathdiff_to_search = 589 # nm
max_index=int(max_pathdiff_to_search*(1754-250)/1744.)

# row of pixels sampled from Michel-Levy chart is in BGR
# pixel to compare should be in RGB

def rgb_to_pathdiff(rgb):
    #rgb=[86,99,134]#664 /668
    #rgb=[161,81,50]#430 /436
    #rgb=[178,77,145]#1101 /1098
    #rgb=[90,146,75]#  /1211
    #rgb=[218,214,45] #/824

    pd_lambda=[]
    for j in range(0,rgb.shape[0]):
        dist=1000000
# For all wavelengths        
#        for i in range(0,ml_pixels_row.shape[0]):
# For only the first order wavelengths (to ca. 920 nm)
        for i in range(0,max_index):
            newdist=(ml_pixels_row[i,0]-rgb[j,2])**2+(ml_pixels_row[i,1]-rgb[j,1])**2+(ml_pixels_row[i,2]-rgb[j,0])**2
            
            if(newdist<dist):
                dist=newdist
                mini=i
                #print(f"j {j} dist {dist} i {i} wl {pathdiff_wavelength[i]} img rgb {rgb[j,0]} {rgb[j,1]} {rgb[j,2]}")
        pd_lambda.append(pathdiff_wavelength[mini])
        #print(f"Wavelength at pixel {rgb[j,:]} for crystal {j} is {pathdiff_wavelength[mini]:.0f} nm")
    return(np.asarray(pd_lambda))


# Test the lookup function in the Michel-Levy table

In [ ]:
# Test the lookup function in the Michel-Levy table
myrgb=np.zeros((2,3))
myrgb[0,:]=[161,81,50]#430 /436
myrgb[1,:]=[143,56,73] # /520

print(rgb_to_pathdiff(myrgb))

# Plot of the pixel intensity vs. path difference of the unmodified color bar chart

In [ ]:
# Be careful, the cv2 format is yx and bgr, not xy and rgb

plt.plot(pathdiff_wavelength, ml_pixels_row[:,0],color="blue")
plt.plot(pathdiff_wavelength, ml_pixels_row[:,1],color="green")
plt.plot(pathdiff_wavelength, ml_pixels_row[:,2],color="red")
plt.plot(pathdiff_wavelength, ml_pixels_row[:,0]+ml_pixels_row[:,1]+ml_pixels_row[:,2], color="gray")


# Plot of the pixel intensity vs. path difference of the histogram-matched color bar chart: chaotic

In [ ]:
# Be careful, the cv2 format is yx and bgr, not xy and rgb

plt.plot(pathdiff_wavelength, matched[1,:,0],color="blue")
plt.plot(pathdiff_wavelength, matched[1,:,1],color="green")
plt.plot(pathdiff_wavelength, matched[1,:,2],color="red")
plt.plot(pathdiff_wavelength, matched[1,:,0]+matched[1,:,1]+matched[1,:,2], color="gray")

# Plot of the histogram of the histogram-matched color bar -- noisy

In [ ]:
color = ('b','g','r')
 
#Loop through each color sequentially
for i,col in enumerate(color):
     
    #To use OpenCV's calcHist function, uncomment below
    #histr = cv2.calcHist([colorimage],[i],None,[256],[0,256])
     
    #To use numpy histogram function, uncomment below
    testimg=mlchart[797:803,250:1800,:]
    testimg=mlchart_shift[797:803,250:1800,:]
    testimg=matched
    testimg2=im_list[5]
    histr1, _ = np.histogram(testimg[:,:,i],256,[0,256],density=True)
    histr2, _ = np.histogram(testimg2[:,:,i],256,[0,256], density=True)
    plt.plot(histr1,color = col)
    plt.plot(histr2,color = col, linestyle="dashed")
    plt.xlim([0,256])
    plt.ylim(0,0.070)
    #fig,ax=plt.subplots(1,2,figsize=(8,8))
    #ax[0].plot(histr1,color = col)  #lookup chart
    #ax[1].plot(histr2,color = col)  #data image
    
     
plt.show()  #Show our plot

# Do the matching of the crystal color to the color in the chart by difference in R,G,B value
### Maybe using normalized intensities would approximate matching the hue of the images?
### Dot-product of the color vectors gave very unexpected results: no input test values returned the correct wavelength shift

In [ ]:
from skimage import measure
import pandas as pd

props_batch=measure.regionprops_table(labels, intensity_image=im_list[0], properties=["label",
                                          "area", "equivalent_diameter",
                                          "mean_intensity", "solidity"])

rgb=np.zeros((len(props_batch["label"]),3))
# In object rgb: red 0, green 1, blue 2
# In df red is 2, green 1, blue 0
rgb[:,0]=props_batch["mean_intensity-2"]
rgb[:,1]=props_batch["mean_intensity-1"]
rgb[:,2]=props_batch["mean_intensity-0"]
props_batch["pathdiff0"]=rgb_to_pathdiff(rgb)
props_batch["az0"]=angles[0]
props_batch["centroid_X"]=centroids[1:,0]
props_batch["centroid_Y"]=centroids[1:,1]

del props_batch["mean_intensity-0"]
del props_batch["mean_intensity-1"]
del props_batch["mean_intensity-2"]

# Make a dataframe for later manipulation
df = pd.DataFrame(props_batch)

for i in range(1,len(im_list)):
    props_img=measure.regionprops_table(labels,intensity_image=im_list[i], properties=["label", "mean_intensity"])
    
    rgb[:,0]=props_img["mean_intensity-2"]
    rgb[:,1]=props_img["mean_intensity-1"]
    rgb[:,2]=props_img["mean_intensity-0"]
    props_img["pathdiff"+str(i)]=rgb_to_pathdiff(rgb)
    props_img["az"+str(i)]=-1*angles[i]
    
    del props_img["mean_intensity-0"]
    del props_img["mean_intensity-1"]
    del props_img["mean_intensity-2"]
    del props_img["label"]

    df_img=pd.DataFrame(props_img)

# concatenate props to props_batch df with key "label"
    df=pd.concat([df, df_img,], axis=1)


In [ ]:
rgb.shape

In [ ]:
df.shape

In [ ]:
# Drop the segments that are too small or too large or otherwise marked for filtration
print(filter_index), len(filter_index)

In [ ]:
# Must subtract one from filter_index to account for zero centroid which is the background (added to the df)
filter_index_minus_one=[x-1 for x in filter_index]

# Check: YES the measures.regioprop function displaces the ROI index by one because it adds the region "0" for the background

In [ ]:
df.loc[filter_index_minus_one[0],:]

In [ ]:
df=df.drop(filter_index_minus_one)

In [ ]:
### Remove the segments corresponding to the mask above (filter_index)
df.head()

# Find the maximum path difference of each crystal and add this variable to the df


In [ ]:
pathdifflist=[]
for i in range(1,len(im_list)):
    pathdifflist.append("pathdiff"+str(i))
print(pathdifflist)

In [ ]:
# df doesn't accept the list of column names generated above so cut and paste the output of the previous cell here:
df["path_diff"]=df[['pathdiff1', 'pathdiff2', 'pathdiff3', 'pathdiff4', 'pathdiff5']].max(axis=1) # new variable is maximum path difference --> calcualte polar angle


In [ ]:
df.head()

# Find the angle of the darkest appearance of each crystal and add this variable to df

In [ ]:
# df["extinction_az"]=df[[pathdifflist]].min(axis=1) # new variable is azimuth at minimum path difference --> roughly indicates azimuth angle (interpolation necessary=
# use .loc to get the index of the column of min and to select corresponding azimuth column

# Make a new reduced df with only label, shape, etc.,  wavelength and corresponding angle of brightest and darkest

# Calculate polar angle and add to reduced df

In [ ]:
ne	= 1.310893
sample_temp = -16.0 # deg C
nw =1.309-(11.4+3.84*sample_temp)*0.00001
largest_sample_path_diff = df["path_diff"].max() # nm
sample_thickness = (largest_sample_path_diff / (ne - nw))*0.000000001*1000 # mm
df["ne_prime"]= (df["path_diff"]/sample_thickness)*0.000001 + nw
df["polar_angle"] = np.arcsin((((nw/df["ne_prime"])**2.-1.)/((nw/ne)**2.-1.))**0.5)*180./np.pi
print(f"max_pd {largest_sample_path_diff:.0f}, thickness {sample_thickness:.2f}")

In [ ]:
print(df.loc[df["path_diff"] == largest_sample_path_diff])

In [ ]:
df.head()

# Plot polar angle, maybe azimuth, report average and std dev
## Expect a random distribution of polar angles to correspond to the directions on the surface of a sphere: the average angle is the center of mass of the spherical shell at 60 degrees from vertical

In [ ]:
garbage=plt.hist(df["area"], bins=20)
print("Average Area and StdDev")
print(df["area"].mean(), np.std(df["area"]))
print("Number of valid crystals")
print(len(df["polar_angle"]))

In [ ]:
# setting up the axes
fig = plt.figure(figsize=(8,8))
ax  = fig.add_subplot(111)
# now plot
data=df["polar_angle"]
myHist = ax.hist(data, 20)
avg=np.linspace(0,int(myHist[0].max()))
x = np.zeros(50)+60.
h = ax.plot(x, avg, lw=2)
# show
plt.show()
print("Average Polar Angle and StdDev")
print(df["polar_angle"].mean(), np.std(df["polar_angle"]))
print("Number of valid crystals")
print(len(df["polar_angle"]))

In [ ]:
myHist[0].max()

# Gap in path difference from 500-610 and 770-830 circa and correspondingly no angles in this range, specifically 50-60 degrees
- This is where blue is highest intensity in the color chart, red is low, green is medium

In [ ]:
plt.plot(df["path_diff"],df["polar_angle"],marker=".", linestyle="none")

In [ ]:
#import cv2
#from matplotlib import pyplot as plt
#import numpy as np
#import imageio.v3 as iio
#from PIL import Image

#import os

#from matplotlib import animation
#plt.rcParams["animation.html"] = "jshtml"  # for matplotlib 2.1 and above uses JavaScript

In [ ]:
# set path containing data folder or use default for Colab (/gdrive/My Drive)
#local_folder = "../"
#import urllib.request
#urllib.request.urlretrieve("https://raw.githubusercontent.com/guiwitz/MLCV/main/notebooks/check_colab.py", "check_colab.py")
#from check_colab import set_datapath
#colab, datapath = set_datapath(local_folder)
#archive = "data/SnowProject/all/"
#experiment = "L7"
#basepath = datapath / archive / experiment
print(basepath)
#imgpath = basepath / "cropped/run3_optimized_cellpose_WB"
print(imgpath)

# Pachitariu, M. & Stringer, C. (2022). Cellpose 2.0: how to train your own model. Nature methods.

conda activate cellpose

python -m cellpose

model results under run3 used the cyto model with the calculated average cell diameter of 28.2 pixels (default pre-training value is 30) and the segmentation channel 1 gray, channel 2 none -- all default values

without training a new model, use cyto model with flow_threshold higher than default of 0.4 and cellprob_threshold lower than default of 0.0. For example ft=0.8 to 0.95 (up to 3) and cp= -4.0

In [ ]:
#import numpy as np
from cellpose import plot, utils
dat = np.load(str(imgpath) + "/4c_seg.npy", allow_pickle=True).item()

# plot image with masks overlaid
mask_RGB = plot.mask_overlay(dat['img'], dat['masks'],
                        colors=np.array(dat['colors']))

# plot image with outlines overlaid in red
outlines = utils.outlines_list(dat['masks'])
plt.imshow(dat['img'])
for o in outlines:
    plt.plot(o[:,0], o[:,1], color='r')

In [ ]:
dat.keys()

In [ ]:
dat["outlines"].max()

# Each mask and each outline object contain integers indicating the correspondence of a pixel to the ID of a segment
## Each image has been processed independently and was divided into segments differently from the other images
## Thus there is a different number of segments for each image and each crystal in each data image has a different ID number

In [ ]:
plt.imshow(dat["masks"])

### Add the mask layers (here, "outlines") together to make one composite mask layer
### This trick does not work if we use the segmentation directly -- only if we use the outlines in a subsequent connected-component analysis
### The connected-component analysis based on this summed outline mask did not work well (very few components were identified)

In [ ]:
#import os
seg_npy_list =[] # list of raw data images

# iterate through files and add to list of images
for file in os.listdir(imgpath):
    if file.endswith("npy"):
        file_path=imgpath / file
    #    print(file_path)
    #load_image(file_path)
        seg_npy_list.append(np.load(file_path, allow_pickle=True).item())

#dat = np.load(str(imgpath) + "/0c_seg.npy", allow_pickle=True).item()

# plot image with masks overlaid
edges=seg_npy_list[0]["outlines"]
#mask=seg_npy_list[0]["masks"]
#labels=seg_npy_list[0]["masks"]
for i in range(0,len(seg_npy_list)):
    edges=edges+seg_npy_list[i]["outlines"]
    print(i, seg_npy_list[i]["filename"],seg_npy_list[i]["outlines"].max())
    #print(f"file seg_npy_list{i} max segments {(seg_npy_list[i]["outlines"]).max()}")
    #mask=mask+seg_npy_list[i]["masks"]

edges=np.clip(edges,0,1)
#mask=np.clip(mask,0,1)
#mask_RGB = plot.mask_overlay(dat['img'], dat['masks'],
#                        colors=np.array(dat['colors']))

# plot image with outlines overlaid in red

#outlines = utils.outlines_list(dat['masks'])
#plt.imshow(dat['img'])
#for o in outlines:
#    plt.plot(o[:,0], o[:,1], color='r')

In [ ]:
%%capture
fig, ax = plt.subplots(figsize=(10,7))
backimg=seg_npy_list[0]["outlines"]
ax.imshow(backimg)
img=plt.imshow(backimg)


def animate(i):
    backimg=seg_npy_list[i]["outlines"]
    imgnew=img.set_data(backimg)

    ax.imshow(backimg)
    ax.set_title(i)

    return(img)

ani = animation.FuncAnimation(fig, animate, frames = 6)

In [ ]:
ani

In [ ]:
#crystal_edges_hand = cv2.imread(str(datapath / "project/L7_1993_crop.jpg"))
fig, ax = plt.subplots(1,2,figsize=(20,24))
ax[0].imshow(edges)
ax[1].imshow(crystal_edges_hand)

#plt.imshow(edges)

# Tracking a crystal across images

- There is a EPFL program in Java called "TrackMate" (ZeroCostDL4Mic) that tracks cells segmented in CellPose as they move in image to image

- Converting the masks and outlines from CellPose _seg.npy object to something sckikit can use in .measure : save mask layer ("outlines" in CellPose jargon) as np array or jpg. Read into "EdgeDetectSegmentation" and use in place of hedsum edge image. The rest of teh processing is the same

# Continue analysis as with HED. As a test, use the edge mask from the first image to generate labels with connected components

In [ ]:
#mask = seg_npy_list[0]['outlines']
mask=edges

In [ ]:
print(mask.max(), mask.min(), mask.size)

In [ ]:
# For OpenCV connected component module the mask must be maximum 255 (feature or segment) and minimum 0 (background)
#thresh=(255*mask/mask.max()).astype("uint8")
thresh=np.clip(mask,0,1)
thresh=(thresh*255).astype("uint8")

In [ ]:
thresh=255-thresh

In [ ]:
blur = cv2.bilateralFilter(thresh,9,5,10)

In [ ]:
thresh5 = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY,11,0)

In [ ]:
type(thresh), thresh.shape,(thresh<100).sum()/(thresh.shape[0]*thresh.shape[1])

In [ ]:
n_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(thresh, connectivity=4)#4 or 8

# OpenCV generates too many connected components based on the additive "outlines" object from cellpose
# Improvement would require smoothing of boundaries, blurring
# It is better to use the cellpose components in the "masks" object to continue

In [ ]:
colors = np.random.randint(0, 255, size=(n_labels, 3), dtype=np.uint8)
colors[0] = [0, 0, 0]  # black background
false_colors = colors[labels]
fig,ax=plt.subplots(1,2,figsize=(20,24))
ax[0].imshow(false_colors)
ax[1].imshow(crystal_edges_hand)

# Create false color image with black background and colored objects
## DO NOT USE the OpenCV connected components above; use the CellPose mask instead
## Since each cellpose mask is different for each image, use the "best" one for all images
## The "masks" object is used to define "labels" which group pixels in a data frame of the regions of interest

In [ ]:
labels=seg_npy_list[1]["masks"]
n_labels=labels.max()+1

In [ ]:
print(n_labels,labels.shape)

In [ ]:
colors = np.random.randint(0, 255, size=(n_labels, 3), dtype=np.uint8)
colors[0] = [0, 0, 0]  # black background
false_colors = colors[labels]
fig,ax=plt.subplots(1,2,figsize=(20,24))
ax[0].imshow(false_colors)
ax[1].imshow(crystal_edges_hand)
#plt.imshow(false_colors)


In [ ]:
print(n_labels)

In [ ]:
# Read in the original images, extract statistics of the crystals with the mask from CellPose
angles = [0.,-16.,-29,-38.5,-60.,-80,]# L7 Irfanview 2023 -- image 2 didn't register well and is omitted
#im_list =[] # list of raw data images

# iterate through files and add to list of images
#for file in os.listdir(imgpath):
#    if file.endswith("jpg"):
#        file_path=imgpath / file
#        print(file_path)
#    #load_image(file_path)
#        im_list.append(cv2.imread(str(file_path)))
#processed_im_list=im_list


In [ ]:

%%capture
fig, ax = plt.subplots(figsize=(10,7))
#fig=plt.figure()
backimg=processed_im_list[0]
ax.imshow(backimg)
ax.imshow(edges, alpha=0.5)
ax.set_axis_off()
img=plt.imshow(backimg)
#img=plt.imshow(img_rgb)


def animate(i):
    backimg=im_list[i]
    imgnew=img.set_data(backimg)

 
    ax.imshow(backimg)
    ax.imshow(edges, alpha=0.5)
    ax.set_axis_off()

    return(img)

ani = animation.FuncAnimation(fig, animate, frames = 6)

In [ ]:
ani

In [ ]:
# Inspect the labelled segments (ROI) with an overlay over an original image
fig, ax = plt.subplots(figsize=(15,15))
ax.imshow(processed_im_list[0])
ax.imshow(false_colors, alpha=0.5)
ax.set_axis_off()


# We can use regionprops from skimage to extract various parameters

- module calculates useful parameters for each object in one or a batch of images.

- first, define the function to convert RGB to path difference (wavelength in nm)

- make a props object for each image in image list
- add features: azimuth angle, path difference wavelength
- drop superfluous variables
- add the coordinates (centroid x,y) to the dataframe --> easiest to do this before filtering
- concatenate and make pandas df

In [ ]:
from skimage import measure
import pandas as pd

props_batch=measure.regionprops_table(labels, intensity_image=im_list[0], properties=["label",
                                          "area", "equivalent_diameter",
                                          "mean_intensity", "solidity"])

rgb=np.zeros((len(props_batch["label"]),3))
rgb[:,0]=props_batch["mean_intensity-2"]
rgb[:,1]=props_batch["mean_intensity-1"]
rgb[:,2]=props_batch["mean_intensity-0"]
props_batch["pathdiff0"]=rgb_to_pathdiff(rgb)
props_batch["az0"]=angles[0]
#props_batch["centroid_X"]=centroids[1:,0]
#props_batch["centroid_Y"]=centroids[1:,1]

del props_batch["mean_intensity-0"]
del props_batch["mean_intensity-1"]
del props_batch["mean_intensity-2"]

# Make a dataframe for later manipulation
df2 = pd.DataFrame(props_batch)

for i in range(1,len(im_list)):
    props_img=measure.regionprops_table(labels,intensity_image=im_list[i], properties=["label", "mean_intensity"])
    
    rgb[:,0]=props_img["mean_intensity-2"]
    rgb[:,1]=props_img["mean_intensity-1"]
    rgb[:,2]=props_img["mean_intensity-0"]
    props_img["pathdiff"+str(i)]=rgb_to_pathdiff(rgb)
    props_img["az"+str(i)]=-1*angles[i]
    
    del props_img["mean_intensity-0"]
    del props_img["mean_intensity-1"]
    del props_img["mean_intensity-2"]
    del props_img["label"]

    df_img=pd.DataFrame(props_img)

# concatenate props to props_batch df with key "label"
    df2=pd.concat([df2, df_img,], axis=1)


In [ ]:
df2.shape

In [ ]:
df2.head()

# Find the maximum path difference of each crystal and add this variable to the df


In [ ]:
pathdifflist=[]
for i in range(1,len(processed_im_list)):
    pathdifflist.append("pathdiff"+str(i))
print(pathdifflist)

In [ ]:
# df doesn't accept the list of column names generated above so cut and paste the output of the previous cell here:
df2["path_diff"]=df2[['pathdiff1', 'pathdiff2', 'pathdiff3', 'pathdiff4', 'pathdiff5']].max(axis=1) # new variable is maximum path difference --> calcualte polar angle


In [ ]:
df2.head()

# Find the angle of the darkest image of each crystal and add this variable to df

In [ ]:
# df["extinction_az"]=df[[pathdifflist]].min(axis=1) # new variable is azimuth at minimum path difference --> roughly indicates azimuth angle (interpolation necessary=
# use .loc to get the index of the column of min and to select corresponding azimuth column

# Make a new reduced df with only label, shape, etc.,  wavelength and corresponding angle of brightest and darkest

# Calculate polar angle and add to reduced df

In [ ]:
ne	= 1.310893
sample_temp = -16.0 # deg C
nw =1.309-(11.4+3.84*sample_temp)*0.00001
largest_sample_path_diff = df2["path_diff"].max() # nm
sample_thickness = (largest_sample_path_diff / (ne - nw))*0.000000001*1000 # mm
df2["ne_prime"]= (df2["path_diff"]/sample_thickness)*0.000001 + nw
df2["polar_angle"] = np.arcsin((((nw/df2["ne_prime"])**2.-1.)/((nw/ne)**2.-1.))**0.5)*180./np.pi
print(f"max_pd {largest_sample_path_diff:.0f}, thickness {sample_thickness:.2f}")

In [ ]:
df2.head()

# Plot polar angle, maybe azimuth, report average and std dev

In [ ]:
garbage=plt.hist(df2["area"], bins=20)
print("Average Area and StdDev")
print(df2["area"].mean(), np.std(df2["area"]))
print("Number of valid crystals")
print(len(df2["polar_angle"]))

In [ ]:
# setting up the axes
fig = plt.figure(figsize=(8,8))
ax  = fig.add_subplot(111)
# now plot
data=df2["polar_angle"]
myHist = ax.hist(data, 20)
avg=np.linspace(0,int(myHist[0].max()))
x = np.zeros(50)+60.
h = ax.plot(x, avg, lw=2)
# show
plt.show()
print("Average Polar Angle and StdDev")
print(df2["polar_angle"].mean(), np.std(df2["polar_angle"]))
print("Number of valid crystals")
print(len(df2["polar_angle"]))

# There are no path differences measured between 410-650 and 730-max (exclusive) and therefore gaps in the angles, too, esp. 50-60 (70) degrees

In [ ]:
plt.plot(df2["path_diff"],df2["polar_angle"],marker=".", linestyle="none")

# Summary
- Polar angles of 42 +/- 11 (302) and 38 +/- 10 (511) with sample thickness of and 0.36 and 0.29 mm versus a random expectation of 60 degrees
- 1993 result for this sample was 47 +/- 16 (157) at 0.30 mm
<br><br>

- Detecting edges first, then thresholding, then segmenting with "connected components" relies on a good thresholding filter
- The HED model is sensitive to the contrast in the input image
- The combination of HED/connected components found a high number of very small crystals -- at least the edge detection identified them as small (thick edge boundaries took away pixels)
- The microbiology-trained cellpose model finds fewer very small shapes but more individual segments
- The labelling in cellpose is different in each image and could be tracked with a bit more work
- However aggregating the cellpose results across images is not straightforward, so you choose the best one <br><br>

- Both methods harvested many many more crystals with less work than by hand
- It's likely that both ML methods overlooked the darkest crystals, which happen to be of highest importance
- It's likely that the human method over-sampled the dark crystals, because they're the most important!
- The ML methods however both found a lower average polar angle which is statistically significantly different from the expected random distribution
- The human method found a larger polar angle which was not distinguishable from random
- This result is due to the sampling of the crystals <br><br>

- Matching path differences from the color chart in the difference algorithm works for pixels chosen from the chart, but does it really match the sample images well?
- The yellows in the data seem to match the yellows in the color chart beyond 750 nm but *there is no blue at all* in the sample so the yellows in the sample are actually bronze and to the left of the blue band in the color chart (smaller path difference) which is how I identified them in 1993 as well.
- Not limiting the Michel-Levy search to order 1 increases variance as some matches fit higher orders
- Samples thicker than order 1 will cause difficulty with this method (using average pixel values in a ROI cannot discern order) 
- Some versions of the Michel-Levy chart show brighter yellows in the smaller path differences ... a matter of (human) judgment
- Using the same CCD/software to calibrate a digital interference chart and record the sample's colors would help this

In [ ]:

fig, ax = plt.subplots(1,2,figsize=(20,100))
ax[0].imshow(spectral_img, extent=[0, 1796, 0, 20*spectral_img.shape[0]])
ax[0].plot(xx, 0*xx, '-', linewidth=0, color="black")
ax[0].set_xlabel("Path Difference (nm)")
ax[0].set_xlim(0,1800)


ax[1].imshow(cv2.cvtColor(im_list[0], cv2.COLOR_BGR2RGB))

# Do snow crystals point straight up after you ski over them? Think about it next time you go skiing!

In [ ]:
lastimage=plt.imread("../project/Colbeck_Equation.jpg")
plt.figure(figsize=(12,12))
plt.axis("off")
plt.imshow(lastimage)

# References
## Ice physics and mineralogy
Colbeck, S.C. Friction of Sliders on Snow. CRREL 92-2. 1992.
<br>
Hobbs. P.V. Ice Physics. Clarendon Press. Oxford. 1964.
<br>
Ackerson, B.J. and N.A. Clark. "Shear-Induced Melting." Phys. Rev. Letters V46#2 1/12/81 pp.123-126.
<br>
Various contributors. Geological Digressions, Optical Mineralogy, some terminology at  https://www.geological-digressions.com/optical-mineralogy-some-terminology/ (03.2023)
<br>
Nikon camera. Microscopy. https://www.microscopyu.com/techniques/polarized-light/principles-of-birefringence (03.2023)
<br><br>
## Michel-Lévy color chart of interference colors
Delly, J.G. The Michel-Lévy Interference Color Chart- Microscopy's Magical Color Key. https://www.mccrone.com/mm/the-michel-levy-interference-color-chart/
<br><br>
## Python to convert RGB into wavelength
<br>
https://stackoverflow.com/questions/71977306/how-to-convert-rgb-to-wavelength-in-python
<br>
https://en.wikipedia.org/wiki/Dominant_wavelength
<br>
https://scipython.com/blog/converting-a-spectrum-to-a-colour/
<br>
https://stackoverflow.com/questions/5817474/how-to-get-the-wavelength-of-a-pixel-using-rgb
<br><br>
## Computer Vision
<br>
Lucas von Chamier*, Romain F. Laine*, Johanna Jukkala, Christoph Spahn, Daniel Krentzel, Elias Nehme, Martina Lerche, Sara Hernández-pérez, Pieta Mattila, Eleni Karinou, Séamus Holden, Ahmet Can Solak, Alexander Krull, Tim-Oliver Buchholz, Martin L Jones, Loic Alain Royer, Christophe Leterrier, Yoav Shechtman, Florian Jug, Mike Heilemann, Guillaume Jacquemet, Ricardo Henriques. Democratising deep learning for microscopy with ZeroCostDL4Mic. Nature Communications, 2021. DOI: https://doi.org/10.1038/s41467-021-22518-0
<br>
Saining Xie, Zhuowen Tu,Holistically-Nested Edge Detection, Proceedings of the IEEE International Conference on Computer Vision (ICCV), 2015, pp. 1395-1403 
<br>
Pachitariu, M. & Stringer, C. (2022). Cellpose 2.0: how to train your own model. Nature methods.
